In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [39]:
#기존 데이터
#association_data = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/7_new_association/9_gold_learning_association.tsv")
#descriptor_data = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_descriptor_combined.tsv")
df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_descriptor_combined.tsv")
x_whole_data = df[df.columns.values[3:].tolist()].values
y_whole_data = df["association"].values

In [23]:
# independent data
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")

In [2]:
# new descriptor - TN 구분한 data(2017.1.16.) 수정됨. 2017.1.23.
df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/4_gold_descriptor_revised.tsv")
x_whole_data = df[df.columns.values[3:].tolist()].values
y_whole_data = df["association"].values

In [6]:
# new descriptor - TN 구분한 data(2017.1.16.) 수정됨. 2017.1.23. 추가됨 2017.2.1.
df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/4_gold_descriptor_revised_0.5%.tsv")
x_whole_data_0_5 = df[df.columns.values[3:].tolist()].values
y_whole_data_0_5 = df["association"].values

df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/4_gold_descriptor_revised_0.1%.tsv")
x_whole_data_0_1 = df[df.columns.values[3:].tolist()].values
y_whole_data_0_1 = df["association"].values

In [22]:
from sklearn.metrics import roc_curve, auc, average_precision_score

def Random_Forest_10_fold(x_whole_data, y_whole_data, user_estimators):
    print("{}----------------------------------------------------".format(user_estimators))
    skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = 10 (10 fold라서.) data가 nega : pos = 76 : 1이기 때문에 validation 뽑을때 그 비율 지켜서 stratified로 한다.
    whole_accuracy = list()
    whole_auc = list()
    whole_aupr = list()
    whole_confusion_mat = list()
    for training_index, validation_index in skf.split(x_whole_data, y_whole_data):
        x_training_set = x_whole_data[training_index] # 0.9에 해당하는 부분
        y_training_set = y_whole_data[training_index]
        x_validation_set = x_whole_data[validation_index] # 0.1에 해당하는 부분
        y_validation_set = y_whole_data[validation_index]

        #classifier = LogisticRegression(penalty='l1', C=0.0001, n_jobs=-1) #n_jobs = -1이면 모든 node 쓰는것. 신경쓰이면 30개 정도.
        classifier = RandomForestClassifier(n_estimators=user_estimators, n_jobs=-1, class_weight='balanced')
        classifier.fit(x_training_set, y_training_set)
        y_predicted_proba = classifier.predict_proba(x_validation_set) # return값이 probability 각각에 대한 확률 결과값 [[0.8(0에대한 확률),0.2(1에대한 확률)],[0.7,0.3],... ]
        y_predicted_label = classifier.predict(x_validation_set) # return이 label임. 위 확률 결과에서 확률이 높은거에 대한 label을 보여줌.

        current_accuracy = classifier.score(x_validation_set, y_validation_set) # accuracy score를 보여줌.
        fpr, tpr, thresholds = roc_curve(y_validation_set, y_predicted_proba[:, 1], pos_label=1) # powitive label만 본것.
        current_auc = auc(fpr, tpr)
        current_confusion_matrix = confusion_matrix(y_validation_set,
                                                    y_predicted_label)
        current_aupr = average_precision_score(y_validation_set, y_predicted_proba[:, 1])
    
        whole_accuracy.append(current_accuracy)
        whole_auc.append(current_auc)
        whole_aupr.append(current_aupr)
        whole_confusion_mat.append((current_confusion_matrix))
        #print("Accuracy of this fold: {}".format(current_accuracy))
        #print("AUC of this fold: {}".format(current_auc))
        #print("AUPR of this fold: {}".format(current_aupr))
        #print("Confustion matrix of this fold")
        #print(current_confusion_matrix)
        #print("\n")
    print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
    print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
    print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
    print("Confustion matrix")
    print(sum(whole_confusion_mat))
    
    return [whole_accuracy, whole_auc, whole_aupr, whole_confusion_mat]

In [33]:
from sklearn.metrics import roc_curve, auc, average_precision_score

def Random_Forest_independent(x_whole_data, y_whole_data, indep_x, indep_y, user_estimators):
    #indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")
    #indep_x = indep[indep.columns.values[3:].tolist()].values
    #indep_y = indep["association"].values

    #classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
    classifier = RandomForestClassifier(n_estimators=user_estimators, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_whole_data, y_whole_data)
    indep_y_predicted_proba = classifier.predict_proba(indep_x)
    indep_y_predicted_label = classifier.predict(indep_x)

    indep_accuracy = classifier.score(indep_x, indep_y)
    indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
    indep_auc = auc(indep_fpr, indep_tpr)
    indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
    indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

    #print("{}----------------------------------------------------".format(user_estimators))
    #print("Independent Accuracy : {}".format(indep_accuracy))
    #print("Independent AUC : {}".format(indep_auc))
    #print("Independent AUPR : {}".format(indep_aupr))
    #print("Independent Confustion matrix")
    #print(indep_confusion_matirx)
    #print("\t")
    
    return [indep_accuracy, indep_auc, indep_aupr, indep_confusion_matirx]
    

In [4]:
# new descriptor(1%) 사용시 2017.1.16. 수정됨. 2017.1.23.
results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_whole_data, y_whole_data, user_estimators))

120----------------------------------------------------
Accuracy of this fold: 0.992877225867
AUC of this fold: 0.948187568227
AUPR of this fold: 0.870278193018
Confustion matrix of this fold
[[5188    3]
 [  35  109]]


Accuracy of this fold: 0.993439550141
AUC of this fold: 0.936164221195
AUPR of this fold: 0.867024719006
Confustion matrix of this fold
[[5189    2]
 [  33  111]]


Accuracy of this fold: 0.990627928772
AUC of this fold: 0.935350178728
AUPR of this fold: 0.808795786196
Confustion matrix of this fold
[[5188    3]
 [  47   97]]


Accuracy of this fold: 0.991190253046
AUC of this fold: 0.929309408378
AUPR of this fold: 0.827341288302
Confustion matrix of this fold
[[5191    0]
 [  47   97]]


Accuracy of this fold: 0.992499531221
AUC of this fold: 0.934105393643
AUPR of this fold: 0.834454716506
Confustion matrix of this fold
[[5189    1]
 [  39  104]]


Accuracy of this fold: 0.99287455466
AUC of this fold: 0.97070684614
AUPR of this fold: 0.869227052411
Confustion matri

In [34]:
#new descriptor(1%)의 independent test 2017.2.1. Random undersampling 추가. 2017.2.2.

from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values
user_estimators = 800

print('Original dataset shape {}'.format(Counter(indep_y)))
indep_result=Random_Forest_independent(x_whole_data, y_whole_data, indep_x, indep_y, user_estimators)
print("{}----------------------------------------------------".format(user_estimators))
print("Independent Accuracy : {}".format(indep_result[0]))
print("Independent AUC : {}".format(indep_result[1]))
print("Independent AUPR : {}".format(indep_result[2]))
print("Independent Confustion matrix")
print(indep_result[3])

rus = RandomUnderSampler()
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for i in range(0, 10):
    x_resampled, y_resampled = rus.fit_sample(indep_x, indep_y)
    indep_result = Random_Forest_independent(x_whole_data, y_whole_data, x_resampled, y_resampled, user_estimators)
    whole_accuracy.append(indep_result[0])
    whole_auc.append(indep_result[1])
    whole_aupr.append(indep_result[2])
    whole_confusion_mat.append(indep_result[3])
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat)) 

Original dataset shape Counter({0: 2462, 1: 93})
800----------------------------------------------------
Independent Accuracy : 0.553033268102
Independent AUC : 0.719877623752
Independent AUPR : 0.236637208478
Independent Confustion matrix
[[1341 1121]
 [  21   72]]
Accuracy avg : 0.654301075269
AUC avg : 0.72488148919
AUPR avg : 0.744254729322
Confustion matrix
[[497 433]
 [210 720]]


In [7]:
# new descriptor(0.5%) 사용시 2017.1.16. 수정됨. 2017.1.23. 추가됨 2017.2.1.
results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_whole_data_0_5, y_whole_data_0_5, user_estimators))

120----------------------------------------------------
Accuracy of this fold: 0.991916310033
AUC of this fold: 0.928362395242
AUPR of this fold: 0.822635184279
Confustion matrix of this fold
[[6162    3]
 [  48   96]]


Accuracy of this fold: 0.990965287684
AUC of this fold: 0.914655988105
AUPR of this fold: 0.752347089351
Confustion matrix of this fold
[[6160    5]
 [  52   92]]


Accuracy of this fold: 0.992074813758
AUC of this fold: 0.935135059025
AUPR of this fold: 0.799808115459
Confustion matrix of this fold
[[6161    4]
 [  46   98]]


Accuracy of this fold: 0.992233317483
AUC of this fold: 0.939443092728
AUPR of this fold: 0.809900911342
Confustion matrix of this fold
[[6163    2]
 [  47   97]]


Accuracy of this fold: 0.990963855422
AUC of this fold: 0.925894543413
AUPR of this fold: 0.760447617707
Confustion matrix of this fold
[[6158    7]
 [  50   93]]


Accuracy of this fold: 0.991280913126
AUC of this fold: 0.91625859947
AUPR of this fold: 0.766581363969
Confustion matr

In [38]:
#new descriptor(0.5%)의 independent test 2017.2.1. Random undersampling 추가. 2017.2.2.
from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values
user_estimators = 300
print('highest AUPR-----------------------------------')
print('Original dataset shape {}'.format(Counter(indep_y)))
indep_result=Random_Forest_independent(x_whole_data_0_5, y_whole_data_0_5, indep_x, indep_y, user_estimators)
print("{}----------------------------------------------------".format(user_estimators))
print("Independent Accuracy : {}".format(indep_result[0]))
print("Independent AUC : {}".format(indep_result[1]))
print("Independent AUPR : {}".format(indep_result[2]))
print("Independent Confustion matrix")
print(indep_result[3])

print('\nResampled dataset')
rus = RandomUnderSampler()
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for i in range(0, 10):
    x_resampled, y_resampled = rus.fit_sample(indep_x, indep_y)
    indep_result = Random_Forest_independent(x_whole_data_0_5, y_whole_data_0_5, x_resampled, y_resampled, user_estimators)
    whole_accuracy.append(indep_result[0])
    whole_auc.append(indep_result[1])
    whole_aupr.append(indep_result[2])
    whole_confusion_mat.append(indep_result[3])
print('resampled dataset shape {}'.format(Counter(y_resampled)))
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat)) 

print('\nhighest AUC-----------------------------------')
user_estimators = 1200
indep_result=Random_Forest_independent(x_whole_data_0_5, y_whole_data_0_5, indep_x, indep_y, user_estimators)
print("{}----------------------------------------------------".format(user_estimators))
print("Independent Accuracy : {}".format(indep_result[0]))
print("Independent AUC : {}".format(indep_result[1]))
print("Independent AUPR : {}".format(indep_result[2]))
print("Independent Confustion matrix")
print(indep_result[3])

print('\nResampled dataset')
rus = RandomUnderSampler()
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for i in range(0, 10):
    x_resampled, y_resampled = rus.fit_sample(indep_x, indep_y)
    indep_result = Random_Forest_independent(x_whole_data_0_5, y_whole_data_0_5, x_resampled, y_resampled, user_estimators)
    whole_accuracy.append(indep_result[0])
    whole_auc.append(indep_result[1])
    whole_aupr.append(indep_result[2])
    whole_confusion_mat.append(indep_result[3])
print('resampled dataset shape {}'.format(Counter(y_resampled)))
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat)) 

highest AUPR-----------------------------------
Original dataset shape Counter({0: 2462, 1: 93})
300----------------------------------------------------
Independent Accuracy : 0.578082191781
Independent AUC : 0.67306718028
Independent AUPR : 0.142314761791
Independent Confustion matrix
[[1405 1057]
 [  21   72]]

Resampled dataset
resampled dataset shape Counter({0: 93, 1: 93})
Accuracy avg : 0.681720430108
AUC avg : 0.676517516476
AUPR avg : 0.673284456005
Confustion matrix
[[548 382]
 [210 720]]

highest AUC-----------------------------------
1200----------------------------------------------------
Independent Accuracy : 0.578082191781
Independent AUC : 0.665884891207
Independent AUPR : 0.140541882556
Independent Confustion matrix
[[1405 1057]
 [  21   72]]

Resampled dataset
Accuracy avg : 0.676344086022
AUC avg : 0.675101167765
AUPR avg : 0.663178694117
Confustion matrix
[[538 392]
 [210 720]]


In [8]:
# new descriptor(0.1%) 사용시 2017.1.16. 수정됨. 2017.1.23. 추가됨 2017.2.1.
results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_whole_data_0_1, y_whole_data_0_1, user_estimators))

120----------------------------------------------------
Accuracy of this fold: 0.990631221455
AUC of this fold: 0.867244474224
AUPR of this fold: 0.633827461405
Confustion matrix of this fold
[[8385   10]
 [  70   74]]


Accuracy of this fold: 0.988991685209
AUC of this fold: 0.895983058699
AUPR of this fold: 0.63692067905
Confustion matrix of this fold
[[8374   21]
 [  73   71]]


Accuracy of this fold: 0.991802318773
AUC of this fold: 0.910508487195
AUPR of this fold: 0.729145137431
Confustion matrix of this fold
[[8379   16]
 [  54   90]]


Accuracy of this fold: 0.990865440918
AUC of this fold: 0.898031235524
AUPR of this fold: 0.706674321347
Confustion matrix of this fold
[[8380   15]
 [  63   81]]


Accuracy of this fold: 0.991215741391
AUC of this fold: 0.885929853351
AUPR of this fold: 0.676472431489
Confustion matrix of this fold
[[8382   13]
 [  62   81]]


Accuracy of this fold: 0.99133286484
AUC of this fold: 0.88130338988
AUPR of this fold: 0.690547878516
Confustion matrix

In [37]:
#new descriptor(0.1%)의 independent test 2017.2.1. Random undersampling 추가. 2017.2.2.

from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values
user_estimators = 1200

print('Original dataset shape {}'.format(Counter(indep_y)))
indep_result=Random_Forest_independent(x_whole_data_0_1, y_whole_data_0_1, indep_x, indep_y, user_estimators)
print("{}----------------------------------------------------".format(user_estimators))
print("Independent Accuracy : {}".format(indep_result[0]))
print("Independent AUC : {}".format(indep_result[1]))
print("Independent AUPR : {}".format(indep_result[2]))
print("Independent Confustion matrix")
print(indep_result[3])

rus = RandomUnderSampler()
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for i in range(0, 10):
    x_resampled, y_resampled = rus.fit_sample(indep_x, indep_y)
    indep_result = Random_Forest_independent(x_whole_data_0_1, y_whole_data_0_1, x_resampled, y_resampled, user_estimators)
    whole_accuracy.append(indep_result[0])
    whole_auc.append(indep_result[1])
    whole_aupr.append(indep_result[2])
    whole_confusion_mat.append(indep_result[3])
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat)) 


Original dataset shape Counter({0: 2462, 1: 93})
1200----------------------------------------------------
Independent Accuracy : 0.824657534247
Independent AUC : 0.797502685988
Independent AUPR : 0.13024938536
Independent Confustion matrix
[[2040  422]
 [  26   67]]
Accuracy avg : 0.765591397849
AUC avg : 0.796097814776
AUPR avg : 0.773235606268
Confustion matrix
[[754 176]
 [260 670]]


In [5]:
#new descriptor(1%)의 independent test 2017.2.1.
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

x_whole_data = df[df.columns.values[3:].tolist()].values
y_whole_data = df["association"].values
indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=800, n_jobs=-1, class_weight='balanced')
classifier.fit(x_whole_data, y_whole_data)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

Independent Accuracy : 0.552641878669
Independent AUC : 0.714160617734
Independent AUPR : 0.174402447238
Independent Confustion matrix
[[1340 1122]
 [  21   72]]
	


In [10]:
#new descriptor(0.5%)의 independent test 2017.2.1.
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1, class_weight='balanced')
classifier.fit(x_whole_data_0_5, y_whole_data_0_5)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("n_estimators = 300 : highest AUPR")
print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

classifier = RandomForestClassifier(n_estimators=1200, n_jobs=-1, class_weight='balanced')
classifier.fit(x_whole_data_0_5, y_whole_data_0_5)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])
print("n_estimators = 1200 : highest AUC")
print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

n_estimators = 300 : highest AUPR
Independent Accuracy : 0.578082191781
Independent AUC : 0.666572766262
Independent AUPR : 0.154087987825
Independent Confustion matrix
[[1405 1057]
 [  21   72]]
	
n_estimators = 1200 : highest AUC
Independent Accuracy : 0.578082191781
Independent AUC : 0.670475092372
Independent AUPR : 0.138136779972
Independent Confustion matrix
[[1405 1057]
 [  21   72]]
	


In [11]:
#new descriptor(0.1%)의 independent test 2017.2.1.
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=1200, n_jobs=-1, class_weight='balanced')
classifier.fit(x_whole_data_0_1, y_whole_data_0_1)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")



Independent Accuracy : 0.822309197652
Independent AUC : 0.805250561219
Independent AUPR : 0.12500479921
Independent Confustion matrix
[[2034  428]
 [  26   67]]
	


In [11]:
#기존 descriptor 사용 2017.1.16.
original_df = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_1_descriptor_combined.tsv")
x_whole_data = original_df[original_df.columns.values[3:].tolist()].values
y_whole_data = original_df["association"].values
results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_whole_data, y_whole_data, user_estimators))

120----------------------------------------------------
Accuracy of this fold: 0.988888888889
AUC of this fold: 0.867375157626
AUPR of this fold: 0.418646630081
Confustion matrix of this fold
[[10902    24]
 [   99    45]]


Accuracy of this fold: 0.988797542687
AUC of this fold: 0.834445080092
AUPR of this fold: 0.371158051324
Confustion matrix of this fold
[[10908    17]
 [  107    37]]


Accuracy of this fold: 0.990604390641
AUC of this fold: 0.847984045258
AUPR of this fold: 0.468445780731
Confustion matrix of this fold
[[10908    17]
 [   87    57]]


Accuracy of this fold: 0.990152678652
AUC of this fold: 0.852038838037
AUPR of this fold: 0.483074736231
Confustion matrix of this fold
[[10905    20]
 [   89    55]]


Accuracy of this fold: 0.988796530538
AUC of this fold: 0.847329055384
AUPR of this fold: 0.401878370227
Confustion matrix of this fold
[[10903    22]
 [  102    41]]


Accuracy of this fold: 0.989067582219
AUC of this fold: 0.872083660047
AUPR of this fold: 0.4229360

In [40]:
#기존 descriptor의 independent test 2017.1.16. #2017.2.13. 수정
from sklearn.metrics import roc_curve, auc, average_precision_score

indep_x = indep.values[:, 3:].astype(float)
indep_y = indep.values[:,2].astype(int)
user_estimators = 800
print('Original dataset shape {}'.format(Counter(indep_y)))
indep_result = Random_Forest_independent(x_whole_data, y_whole_data, indep_x, indep_y, user_estimators)
print("{}----------------------------------------------------".format(user_estimators))
print("Independent Accuracy : {}".format(indep_result[0]))
print("Independent AUC : {}".format(indep_result[1]))
print("Independent AUPR : {}".format(indep_result[2]))
print("Independent Confustion matrix")
print(indep_result[3])

rus = RandomUnderSampler()
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for i in range(0, 10):
    x_resampled, y_resampled = rus.fit_sample(indep_x, indep_y)
    indep_result = Random_Forest_independent(x_whole_data, y_whole_data, x_resampled, y_resampled, user_estimators)
    whole_accuracy.append(indep_result[0])
    whole_auc.append(indep_result[1])
    whole_aupr.append(indep_result[2])
    whole_confusion_mat.append(indep_result[3])
print('\nResampled dataset shape {}'.format(Counter(y_resampled)))
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat)) 

Original dataset shape Counter({0: 2462, 1: 93})
800----------------------------------------------------
Independent Accuracy : 0.979647749511
Independent AUC : 0.903319706856
Independent AUPR : 0.668596110169
Independent Confustion matrix
[[2439   23]
 [  29   64]]

Resampled dataset shape Counter({0: 93, 1: 93})
Accuracy avg : 0.838709677419
AUC avg : 0.879118973292
AUPR avg : 0.914989381119
Confustion matrix
[[920  10]
 [290 640]]


In [29]:
# imbalanced learn RandomUnderSampler사용(기존 descriptor) 2017.1.17.

from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

print('Original dataset shape {}'.format(Counter(y_whole_data)))
rus = RandomUnderSampler()
x_resampled, y_resampled = rus.fit_sample(x_whole_data, y_whole_data)
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_resampled, y_resampled, user_estimators))

Original dataset shape Counter({0: 109251, 1: 1434})
Resampled dataset shape Counter({0: 1434, 1: 1434})
120----------------------------------------------------
Accuracy of this fold: 0.694444444444
AUC of this fold: 0.773992091049
AUPR of this fold: 0.797459860278
Confustion matrix of this fold
[[ 97  47]
 [ 41 103]]


Accuracy of this fold: 0.673611111111
AUC of this fold: 0.747058256173
AUPR of this fold: 0.74763650394
Confustion matrix of this fold
[[98 46]
 [48 96]]


Accuracy of this fold: 0.746527777778
AUC of this fold: 0.824074074074
AUPR of this fold: 0.845831813771
Confustion matrix of this fold
[[103  41]
 [ 32 112]]


Accuracy of this fold: 0.711805555556
AUC of this fold: 0.770568094136
AUPR of this fold: 0.792462629338
Confustion matrix of this fold
[[ 98  46]
 [ 37 107]]


Accuracy of this fold: 0.667832167832
AUC of this fold: 0.729962345347
AUPR of this fold: 0.712530871237
Confustion matrix of this fold
[[95 48]
 [47 96]]


Accuracy of this fold: 0.692307692308
AUC o

In [30]:
# random undersampling (기존 descriptor) independent test 2017.1.17. - 

indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=1200, n_jobs=-1, class_weight='balanced')
classifier.fit(x_resampled, y_resampled)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

Independent Accuracy : 0.716634050881
Independent AUC : 0.811644523641
Independent AUPR : 0.250116984496
Independent Confustion matrix
[[1760  702]
 [  22   71]]
	


In [32]:
# imbalanced learn 사용(기존 descriptor) 2017.1.17. - onsidedselection

from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import OneSidedSelection

print('Original dataset shape {}'.format(Counter(y_whole_data)))
oss = OneSidedSelection()
x_resampled, y_resampled = oss.fit_sample(x_whole_data, y_whole_data)
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_resampled, y_resampled, user_estimators))

Original dataset shape Counter({0: 109251, 1: 1434})
Resampled dataset shape Counter({0: 109107, 1: 1434})
120----------------------------------------------------
Accuracy of this fold: 0.990411578471
AUC of this fold: 0.878392982617
AUPR of this fold: 0.488511294447
Confustion matrix of this fold
[[10895    16]
 [   90    54]]


Accuracy of this fold: 0.991044776119
AUC of this fold: 0.863685284473
AUPR of this fold: 0.505906895089
Confustion matrix of this fold
[[10903     8]
 [   91    53]]


Accuracy of this fold: 0.988692899141
AUC of this fold: 0.828177985519
AUPR of this fold: 0.371111925927
Confustion matrix of this fold
[[10893    18]
 [  107    37]]


Accuracy of this fold: 0.989959294437
AUC of this fold: 0.86181503885
AUPR of this fold: 0.501718742975
Confustion matrix of this fold
[[10892    19]
 [   92    52]]


Accuracy of this fold: 0.990953500995
AUC of this fold: 0.878129340186
AUPR of this fold: 0.523941507934
Confustion matrix of this fold
[[10901    10]
 [   90    

In [34]:
# onesided selection under sampling undersampling (기존 descriptor) independent test 2017.1.18. - 

indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=1200, n_jobs=-1, class_weight='balanced')
classifier.fit(x_resampled, y_resampled)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

Independent Accuracy : 0.979256360078
Independent AUC : 0.896912642052
Independent AUPR : 0.659183060549
Independent Confustion matrix
[[2438   24]
 [  29   64]]
	


In [41]:
# imbalanced learn 사용(기존 descriptor) 2017.1.18. - nearmiss

from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import NearMiss

print('Original dataset shape {}'.format(Counter(y_whole_data)))
nm = NearMiss()
x_resampled, y_resampled = nm.fit_sample(x_whole_data, y_whole_data)
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_resampled, y_resampled, user_estimators))

Original dataset shape Counter({0: 109251, 1: 1434})
Resampled dataset shape Counter({0: 1434, 1: 1434})
120----------------------------------------------------
Accuracy of this fold: 0.947916666667
AUC of this fold: 0.979552469136
AUPR of this fold: 0.985241027579
Confustion matrix of this fold
[[143   1]
 [ 14 130]]


Accuracy of this fold: 0.958333333333
AUC of this fold: 0.988498263889
AUPR of this fold: 0.991789296001
Confustion matrix of this fold
[[144   0]
 [ 12 132]]


Accuracy of this fold: 0.96875
AUC of this fold: 0.988522376543
AUPR of this fold: 0.992561907261
Confustion matrix of this fold
[[143   1]
 [  8 136]]


Accuracy of this fold: 0.965277777778
AUC of this fold: 0.980107060185
AUPR of this fold: 0.985754725342
Confustion matrix of this fold
[[142   2]
 [  8 136]]


Accuracy of this fold: 0.961538461538
AUC of this fold: 0.981857303536
AUPR of this fold: 0.98813419351
Confustion matrix of this fold
[[143   0]
 [ 11 132]]


Accuracy of this fold: 0.972027972028
AUC 

In [42]:
# nearmiss under sampling undersampling (기존 descriptor) independent test 2017.1.17. - 

indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

indep_x = indep[indep.columns.values[3:].tolist()].values
indep_y = indep["association"].values

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1, class_weight='balanced')
classifier.fit(x_resampled, y_resampled)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

Independent Accuracy : 0.06614481409
Independent AUC : 0.702259287405
Independent AUPR : 0.212383359166
Independent Confustion matrix
[[  79 2383]
 [   3   90]]
	


In [37]:
# imbalanced learn 사용(기존 descriptor) 2017.1.18. - tomelinks

from sklearn.metrics import roc_curve, auc, average_precision_score
from collections import Counter
from imblearn.under_sampling import TomekLinks

print('Original dataset shape {}'.format(Counter(y_whole_data)))
tl = TomekLinks()
x_resampled, y_resampled = tl.fit_sample(x_whole_data, y_whole_data)
print('Resampled dataset shape {}'.format(Counter(y_resampled)))

results = list()
for user_estimators in [120,300,500,800,1200]:
    results.append(Random_Forest_10_fold(x_resampled, y_resampled, user_estimators))

Original dataset shape Counter({0: 109251, 1: 1434})
Resampled dataset shape Counter({0: 109152, 1: 1434})
120----------------------------------------------------
Accuracy of this fold: 0.989963833635
AUC of this fold: 0.831143950267
AUPR of this fold: 0.451402116108
Confustion matrix of this fold
[[10898    18]
 [   93    51]]


Accuracy of this fold: 0.989059674503
AUC of this fold: 0.856215455906
AUPR of this fold: 0.456391545023
Confustion matrix of this fold
[[10899    17]
 [  104    40]]


Accuracy of this fold: 0.990505470657
AUC of this fold: 0.877790820481
AUPR of this fold: 0.48879491156
Confustion matrix of this fold
[[10905    10]
 [   95    49]]


Accuracy of this fold: 0.988787412967
AUC of this fold: 0.874766503792
AUPR of this fold: 0.371034868318
Confustion matrix of this fold
[[10900    15]
 [  109    35]]


Accuracy of this fold: 0.990775908844
AUC of this fold: 0.893746976798
AUPR of this fold: 0.500509836427
Confustion matrix of this fold
[[10902    13]
 [   89    

In [ ]:
from sklearn.metrics import roc_curve, auc, average_precision_score

skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = 10 (10 fold라서.) data가 nega : pos = 76 : 1이기 때문에 validation 뽑을때 그 비율 지켜서 stratified로 한다.
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for training_index, validation_index in skf.split(x_whole_data, y_whole_data):
    x_training_set = x_whole_data[training_index] # 0.9에 해당하는 부분
    y_training_set = y_whole_data[training_index]
    x_validation_set = x_whole_data[validation_index] # 0.1에 해당하는 부분
    y_validation_set = y_whole_data[validation_index]

    #classifier = LogisticRegression(penalty='l1', C=0.0001, n_jobs=-1) #n_jobs = -1이면 모든 node 쓰는것. 신경쓰이면 30개 정도.
    classifier = RandomForestClassifier(n_estimators=120, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_training_set, y_training_set)
    y_predicted_proba = classifier.predict_proba(x_validation_set) # return값이 probability 각각에 대한 확률 결과값 [[0.8(0에대한 확률),0.2(1에대한 확률)],[0.7,0.3],... ]
    y_predicted_label = classifier.predict(x_validation_set) # return이 label임. 위 확률 결과에서 확률이 높은거에 대한 label을 보여줌.

    current_accuracy = classifier.score(x_validation_set, y_validation_set) # accuracy score를 보여줌.
    fpr, tpr, thresholds = roc_curve(y_validation_set, y_predicted_proba[:, 1], pos_label=1) # powitive label만 본것.
    current_auc = auc(fpr, tpr)
    current_confusion_matrix = confusion_matrix(y_validation_set,
                                                y_predicted_label)
    current_aupr = average_precision_score(y_validation_set, y_predicted_proba[:, 1])
    
    whole_accuracy.append(current_accuracy)
    whole_auc.append(current_auc)
    whole_aupr.append(current_aupr)
    whole_confusion_mat.append((current_confusion_matrix))
    print("Accuracy of this fold: {}".format(current_accuracy))
    print("AUC of this fold: {}".format(current_auc))
    print("AUPR of this fold: {}".format(current_aupr))
    print("Confustion matrix of this fold")
    print(current_confusion_matrix)
    print("\n")
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat))

In [7]:
from sklearn.metrics import roc_curve, auc, average_precision_score

skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = 10 (10 fold라서.) data가 nega : pos = 76 : 1이기 때문에 validation 뽑을때 그 비율 지켜서 stratified로 한다.
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for training_index, validation_index in skf.split(x_whole_data, y_whole_data):
    x_training_set = x_whole_data[training_index] # 0.9에 해당하는 부분
    y_training_set = y_whole_data[training_index]
    x_validation_set = x_whole_data[validation_index] # 0.1에 해당하는 부분
    y_validation_set = y_whole_data[validation_index]

    #classifier = LogisticRegression(penalty='l1', C=0.001, n_jobs=-1) #n_jobs = -1이면 모든 node 쓰는것. 신경쓰이면 30개 정도.
    classifier = RandomForestClassifier(n_estimators=300, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_training_set, y_training_set)
    y_predicted_proba = classifier.predict_proba(x_validation_set) # return값이 probability 각각에 대한 확률 결과값 [[0.8(0에대한 확률),0.2(1에대한 확률)],[0.7,0.3],... ]
    y_predicted_label = classifier.predict(x_validation_set) # return이 label임. 위 확률 결과에서 확률이 높은거에 대한 label을 보여줌.

    current_accuracy = classifier.score(x_validation_set, y_validation_set) # accuracy score를 보여줌.
    fpr, tpr, thresholds = roc_curve(y_validation_set, y_predicted_proba[:, 1], pos_label=1) # powitive label만 본것.
    current_auc = auc(fpr, tpr)
    current_confusion_matrix = confusion_matrix(y_validation_set,
                                                y_predicted_label)
    current_aupr = average_precision_score(y_validation_set, y_predicted_proba[:, 1])
    
    whole_accuracy.append(current_accuracy)
    whole_auc.append(current_auc)
    whole_aupr.append(current_aupr)
    whole_confusion_mat.append((current_confusion_matrix))
    print("Accuracy of this fold: {}".format(current_accuracy))
    print("AUC of this fold: {}".format(current_auc))
    print("AUPR of this fold: {}".format(current_aupr))
    print("Confustion matrix of this fold")
    print(current_confusion_matrix)
    print("\n")
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat))

Accuracy of this fold: 0.989882565492
AUC of this fold: 0.876410689589
AUPR of this fold: 0.472315142024
Confustion matrix of this fold
[[10907    19]
 [   93    51]]


Accuracy of this fold: 0.989520281868
AUC of this fold: 0.839363081617
AUPR of this fold: 0.414750941914
Confustion matrix of this fold
[[10904    21]
 [   95    49]]


Accuracy of this fold: 0.988797542687
AUC of this fold: 0.851094584287
AUPR of this fold: 0.395496562733
Confustion matrix of this fold
[[10910    15]
 [  109    35]]


Accuracy of this fold: 0.988616857891
AUC of this fold: 0.882013729977
AUPR of this fold: 0.433313248148
Confustion matrix of this fold
[[10898    27]
 [   99    45]]


Accuracy of this fold: 0.989971087821
AUC of this fold: 0.874251652238
AUPR of this fold: 0.451735127441
Confustion matrix of this fold
[[10907    18]
 [   93    50]]


Accuracy of this fold: 0.98942898446
AUC of this fold: 0.881530460386
AUPR of this fold: 0.414880862552
Confustion matrix of this fold
[[10907    18]
 [   

In [8]:
from sklearn.metrics import roc_curve, auc, average_precision_score

skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = 10 (10 fold라서.) data가 nega : pos = 76 : 1이기 때문에 validation 뽑을때 그 비율 지켜서 stratified로 한다.
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for training_index, validation_index in skf.split(x_whole_data, y_whole_data):
    x_training_set = x_whole_data[training_index] # 0.9에 해당하는 부분
    y_training_set = y_whole_data[training_index]
    x_validation_set = x_whole_data[validation_index] # 0.1에 해당하는 부분
    y_validation_set = y_whole_data[validation_index]

    #classifier = LogisticRegression(penalty='l1', C=0.01, n_jobs=-1) #n_jobs = -1이면 모든 node 쓰는것. 신경쓰이면 30개 정도.
    classifier = RandomForestClassifier(n_estimators=500, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_training_set, y_training_set)
    y_predicted_proba = classifier.predict_proba(x_validation_set) # return값이 probability 각각에 대한 확률 결과값 [[0.8(0에대한 확률),0.2(1에대한 확률)],[0.7,0.3],... ]
    y_predicted_label = classifier.predict(x_validation_set) # return이 label임. 위 확률 결과에서 확률이 높은거에 대한 label을 보여줌.

    current_accuracy = classifier.score(x_validation_set, y_validation_set) # accuracy score를 보여줌.
    fpr, tpr, thresholds = roc_curve(y_validation_set, y_predicted_proba[:, 1], pos_label=1) # powitive label만 본것.
    current_auc = auc(fpr, tpr)
    current_confusion_matrix = confusion_matrix(y_validation_set,
                                                y_predicted_label)
    current_aupr = average_precision_score(y_validation_set, y_predicted_proba[:, 1])
    
    whole_accuracy.append(current_accuracy)
    whole_auc.append(current_auc)
    whole_aupr.append(current_aupr)
    whole_confusion_mat.append((current_confusion_matrix))
    print("Accuracy of this fold: {}".format(current_accuracy))
    print("AUC of this fold: {}".format(current_auc))
    print("AUPR of this fold: {}".format(current_aupr))
    print("Confustion matrix of this fold")
    print(current_confusion_matrix)
    print("\n")
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat))

Accuracy of this fold: 0.989430894309
AUC of this fold: 0.868545276812
AUPR of this fold: 0.435892977198
Confustion matrix of this fold
[[10903    23]
 [   94    50]]


Accuracy of this fold: 0.989429939471
AUC of this fold: 0.856746122553
AUPR of this fold: 0.426079076119
Confustion matrix of this fold
[[10910    15]
 [  102    42]]


Accuracy of this fold: 0.988887885084
AUC of this fold: 0.874398677854
AUPR of this fold: 0.407779820746
Confustion matrix of this fold
[[10902    23]
 [  100    44]]


Accuracy of this fold: 0.989791309061
AUC of this fold: 0.878923531655
AUPR of this fold: 0.433465872316
Confustion matrix of this fold
[[10912    13]
 [  100    44]]


Accuracy of this fold: 0.988796530538
AUC of this fold: 0.859357667504
AUPR of this fold: 0.37558570886
Confustion matrix of this fold
[[10906    19]
 [  105    38]]


Accuracy of this fold: 0.990513191182
AUC of this fold: 0.894507369061
AUPR of this fold: 0.507700094844
Confustion matrix of this fold
[[10910    15]
 [   

In [9]:
from sklearn.metrics import roc_curve, auc, average_precision_score

skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = 10 (10 fold라서.) data가 nega : pos = 76 : 1이기 때문에 validation 뽑을때 그 비율 지켜서 stratified로 한다.
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for training_index, validation_index in skf.split(x_whole_data, y_whole_data):
    x_training_set = x_whole_data[training_index] # 0.9에 해당하는 부분
    y_training_set = y_whole_data[training_index]
    x_validation_set = x_whole_data[validation_index] # 0.1에 해당하는 부분
    y_validation_set = y_whole_data[validation_index]

    #classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs=-1) #n_jobs = -1이면 모든 node 쓰는것. 신경쓰이면 30개 정도.
    classifier = RandomForestClassifier(n_estimators=800, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_training_set, y_training_set)
    y_predicted_proba = classifier.predict_proba(x_validation_set) # return값이 probability 각각에 대한 확률 결과값 [[0.8(0에대한 확률),0.2(1에대한 확률)],[0.7,0.3],... ]
    y_predicted_label = classifier.predict(x_validation_set) # return이 label임. 위 확률 결과에서 확률이 높은거에 대한 label을 보여줌.

    current_accuracy = classifier.score(x_validation_set, y_validation_set) # accuracy score를 보여줌.
    fpr, tpr, thresholds = roc_curve(y_validation_set, y_predicted_proba[:, 1], pos_label=1) # powitive label만 본것.
    current_auc = auc(fpr, tpr)
    current_confusion_matrix = confusion_matrix(y_validation_set,
                                                y_predicted_label)
    current_aupr = average_precision_score(y_validation_set, y_predicted_proba[:, 1])
    
    whole_accuracy.append(current_accuracy)
    whole_auc.append(current_auc)
    whole_aupr.append(current_aupr)
    whole_confusion_mat.append((current_confusion_matrix))
    print("Accuracy of this fold: {}".format(current_accuracy))
    print("AUC of this fold: {}".format(current_auc))
    print("AUPR of this fold: {}".format(current_aupr))
    print("Confustion matrix of this fold")
    print(current_confusion_matrix)
    print("\n")
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat))

Accuracy of this fold: 0.987895212285
AUC of this fold: 0.84438908465
AUPR of this fold: 0.326368214106
Confustion matrix of this fold
[[10904    22]
 [  112    32]]


Accuracy of this fold: 0.989971993857
AUC of this fold: 0.864346872616
AUPR of this fold: 0.469312991303
Confustion matrix of this fold
[[10910    15]
 [   96    48]]


Accuracy of this fold: 0.99024302105
AUC of this fold: 0.903686753115
AUPR of this fold: 0.504502370249
Confustion matrix of this fold
[[10908    17]
 [   91    53]]


Accuracy of this fold: 0.989158912278
AUC of this fold: 0.893111174676
AUPR of this fold: 0.453194611269
Confustion matrix of this fold
[[10905    20]
 [  100    44]]


Accuracy of this fold: 0.990332490061
AUC of this fold: 0.890560240675
AUPR of this fold: 0.493945091931
Confustion matrix of this fold
[[10905    20]
 [   87    56]]


Accuracy of this fold: 0.98960968558
AUC of this fold: 0.853993375046
AUPR of this fold: 0.415113333015
Confustion matrix of this fold
[[10908    17]
 [   98

In [10]:
from sklearn.metrics import roc_curve, auc, average_precision_score

skf = StratifiedKFold(n_splits=10, shuffle=True) # n_splits = 10 (10 fold라서.) data가 nega : pos = 76 : 1이기 때문에 validation 뽑을때 그 비율 지켜서 stratified로 한다.
whole_accuracy = list()
whole_auc = list()
whole_aupr = list()
whole_confusion_mat = list()
for training_index, validation_index in skf.split(x_whole_data, y_whole_data):
    x_training_set = x_whole_data[training_index] # 0.9에 해당하는 부분
    y_training_set = y_whole_data[training_index]
    x_validation_set = x_whole_data[validation_index] # 0.1에 해당하는 부분
    y_validation_set = y_whole_data[validation_index]

    #classifier = LogisticRegression(penalty='l1', C=1, n_jobs=-1) #n_jobs = -1이면 모든 node 쓰는것. 신경쓰이면 30개 정도.
    classifier = RandomForestClassifier(n_estimators=1200, n_jobs=-1, class_weight='balanced')
    classifier.fit(x_training_set, y_training_set)
    y_predicted_proba = classifier.predict_proba(x_validation_set) # return값이 probability 각각에 대한 확률 결과값 [[0.8(0에대한 확률),0.2(1에대한 확률)],[0.7,0.3],... ]
    y_predicted_label = classifier.predict(x_validation_set) # return이 label임. 위 확률 결과에서 확률이 높은거에 대한 label을 보여줌.

    current_accuracy = classifier.score(x_validation_set, y_validation_set) # accuracy score를 보여줌.
    fpr, tpr, thresholds = roc_curve(y_validation_set, y_predicted_proba[:, 1], pos_label=1) # powitive label만 본것.
    current_auc = auc(fpr, tpr)
    current_confusion_matrix = confusion_matrix(y_validation_set,
                                                y_predicted_label)
    current_aupr = average_precision_score(y_validation_set, y_predicted_proba[:, 1])
    
    whole_accuracy.append(current_accuracy)
    whole_auc.append(current_auc)
    whole_aupr.append(current_aupr)
    whole_confusion_mat.append((current_confusion_matrix))
    print("Accuracy of this fold: {}".format(current_accuracy))
    print("AUC of this fold: {}".format(current_auc))
    print("AUPR of this fold: {}".format(current_aupr))
    print("Confustion matrix of this fold")
    print(current_confusion_matrix)
    print("\n")
print("Accuracy avg : {}").format(sum(whole_accuracy)/len(whole_accuracy))
print("AUC avg : {}").format(sum(whole_auc)/len(whole_auc))
print("AUPR avg : {}").format(sum(whole_aupr)/len(whole_aupr))
print("Confustion matrix")
print(sum(whole_confusion_mat))

Accuracy of this fold: 0.989972899729
AUC of this fold: 0.891333681636
AUPR of this fold: 0.499149559686
Confustion matrix of this fold
[[10901    25]
 [   86    58]]


Accuracy of this fold: 0.988165145903
AUC of this fold: 0.854820429697
AUPR of this fold: 0.378107366953
Confustion matrix of this fold
[[10897    28]
 [  103    41]]


Accuracy of this fold: 0.989429939471
AUC of this fold: 0.8595032418
AUPR of this fold: 0.415617917788
Confustion matrix of this fold
[[10904    21]
 [   96    48]]


Accuracy of this fold: 0.989971993857
AUC of this fold: 0.871734998729
AUPR of this fold: 0.436236638907
Confustion matrix of this fold
[[10907    18]
 [   93    51]]


Accuracy of this fold: 0.990061438381
AUC of this fold: 0.848808948489
AUPR of this fold: 0.429378993302
Confustion matrix of this fold
[[10913    12]
 [   98    45]]


Accuracy of this fold: 0.988796530538
AUC of this fold: 0.884777007889
AUPR of this fold: 0.430413585412
Confustion matrix of this fold
[[10903    22]
 [  10

In [46]:
indep = pd.read_table("/DAS_Storage1/aschoi/data/Drug_Repositioning/8_new_training/2_indep_asso_descriptor.tsv")

from sklearn.metrics import roc_curve, auc, average_precision_score

x_whole_data = descriptor_data.values
y_whole_data = association_data["association"].values
indep_x = indep.values[:, 3:].astype(float)
indep_y = indep.values[:,2].astype(int)

#classifier = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)
classifier = RandomForestClassifier(n_estimators=800, n_jobs=-1, class_weight='balanced')
classifier.fit(x_whole_data, y_whole_data)
indep_y_predicted_proba = classifier.predict_proba(indep_x)
indep_y_predicted_label = classifier.predict(indep_x)

indep_accuracy = classifier.score(indep_x, indep_y)
indep_fpr, indep_tpr, indep_thresholds = roc_curve(indep_y, indep_y_predicted_proba[:, 1], pos_label = 1)
indep_auc = auc(indep_fpr, indep_tpr)
indep_confusion_matirx = confusion_matrix(indep_y, indep_y_predicted_label)
indep_aupr = average_precision_score(indep_y, indep_y_predicted_proba[:, 1])

print("Independent Accuracy : {}".format(indep_accuracy))
print("Independent AUC : {}".format(indep_auc))
print("Independent AUPR : {}".format(indep_aupr))
print("Independent Confustion matrix")
print(indep_confusion_matirx)
print("\t")

Independent Accuracy : 0.979647749511
Independent AUC : 0.874295746967
Independent AUPR : 0.649091752381
Independent Confustion matrix
[[2439   23]
 [  29   64]]
	
